In [3]:
pip install -U datasets transformers sacrebleu sentencepiece evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 128.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.2
    Uninstalling transformers-4.52.2:
      Successfully uninstalled transformers-4.52.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dep

In [ ]:
import os
from tqdm import tqdm
import datasets
from datasets import load_dataset
import evaluate
from transformers import MarianTokenizer, MarianMTModel, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

In [5]:
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  Tesla T4
Using device: cuda


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
cd /content/drive/MyDrive/IT4772E - NLP/marianmt/

/content/drive/MyDrive/IT4772E - NLP/marianmt


In [13]:
model_name = "Helsinki-NLP/opus-mt-en-vi"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [14]:
drive_train = "/content/drive/MyDrive/IT4772E - NLP/marianmt/train.json"
drive_dev = "/content/drive/MyDrive/IT4772E - NLP/marianmt/dev.json"
drive_test = "/content/drive/MyDrive/IT4772E - NLP/marianmt/test.json"
local_train = "/content/train.json"
local_dev = "/content/dev.json"
local_test = "/content/test.json"

if os.path.exists(drive_train):
    os.system(f'cp "{drive_train}" "{local_train}"')
if os.path.exists(drive_dev):
    os.system(f'cp "{drive_dev}" "{local_dev}"')
if os.path.exists(drive_test):
    os.system(f'cp "{drive_test}" "{local_test}"')

In [ ]:
# Load dataset
train_dataset = load_dataset("json", data_files=local_train, split="train")
dev_dataset = load_dataset("json", data_files=local_dev, split="train")
test_dataset = load_dataset("json", data_files=local_test, split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [17]:
# Hàm kiểm tra độ dài token
def filter_by_length(example):
    en_tokens = len(tokenizer.encode(example["translation"]["en"], add_special_tokens=True))
    vi_tokens = len(tokenizer.encode(example["translation"]["vi"], add_special_tokens=True))
    return en_tokens <= 64 and vi_tokens <= 64

# Lọc dataset
filtered_train = train_dataset.filter(filter_by_length, num_proc=4)
filtered_dev = dev_dataset.filter(filter_by_length, num_proc=4)
filtered_test = test_dataset.filter(filter_by_length, num_proc=4)

Filter (num_proc=4):   0%|          | 0/2964379 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/18645 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/19067 [00:00<?, ? examples/s]

In [18]:
import random

# Chia train và valid từ train_dev
train_indices = random.sample(range(len(filtered_train)), min(200000, len(filtered_train)))
valid_indices = random.sample(range(len(filtered_dev)), min(10000, len(filtered_dev)))
test_indices = random.sample(range(len(filtered_test)), min(5000, len(filtered_test)))
train_dataset = filtered_train.select(train_indices)
valid_dataset = filtered_dev.select(valid_indices)
test_dataset = filtered_test.select(test_indices)

print(f"Số mẫu train: {len(train_dataset)}")
print(f"Số mẫu valid: {len(valid_dataset)}")
print(f"Số mẫu test: {len(test_dataset)}")

Số mẫu train: 200000
Số mẫu valid: 10000
Số mẫu test: 5000


In [19]:
def preprocess(examples):
    sources = [ex["en"] for ex in examples["translation"]]
    targets = [ex["vi"] for ex in examples["translation"]]
    inputs = tokenizer(sources, truncation=True, padding="longest", max_length=64, return_tensors="pt")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, truncation=True, padding="longest", max_length=64, return_tensors="pt")
    inputs["labels"] = labels["input_ids"]
    return inputs

In [20]:
tokenized_train = train_dataset.map(preprocess, batched=True, batch_size=1000)
tokenized_valid = valid_dataset.map(preprocess, batched=True, batch_size=1000)
tokenized_test = test_dataset.map(preprocess, batched=True, batch_size=1000)

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [21]:
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding="longest",
    max_length=64,
    pad_to_multiple_of=8,
    label_pad_token_id=-100,
)

In [22]:
bleu = evaluate.load("sacrebleu")

In [23]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = [[l if l != -100 else tokenizer.pad_token_id for l in label] for label in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [24]:
import os
os.makedirs("./logs/tensorboard", exist_ok=True)

In [25]:
# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./marian_finetuned_vi",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    num_train_epochs=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./logs/tensorboard",  # Thư mục lưu log TensorBoard
    logging_steps=50,
    fp16=True,
    optim="adamw_torch",
    warmup_steps=500,
    max_grad_norm=1.0,
    dataloader_num_workers=4,
    report_to="tensorboard",  # Kích hoạt TensorBoard
    disable_tqdm=False,  # Giữ progress bar
)

In [26]:
# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-26-70a3e9cf1008>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [27]:
# Train
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu
1,0.509400,0.510430,33.142835
2,0.469800,0.499526,33.573481
3,0.430100,0.497348,33.685310
4,0.381200,0.499295,33.681010
5,0.363400,0.501137,33.678003


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[53684]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number 

TrainOutput(global_step=31250, training_loss=0.43868648583984376, metrics={'train_runtime': 11600.8569, 'train_samples_per_second': 86.201, 'train_steps_per_second': 2.694, 'total_flos': 1.4792263401996288e+16, 'train_loss': 0.43868648583984376, 'epoch': 5.0})

In [28]:
# Lưu mô hình
trainer.save_model("./marian_finetuned_vi_final")

In [29]:
def postprocess(preds, labels):
    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return preds, labels

results = trainer.predict(tokenized_test)
preds, labels = postprocess(results.predictions, results.label_ids)

bleu_score = bleu.compute(predictions=preds, references=[[l] for l in labels])
print(f"BLEU: {bleu_score['score']:.2f}")

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


BLEU: 32.52
